In [1]:
# Instalação de pacotes
#!pip install requests
#!pip install bs4
#!pip install lxml
#!pip install tqdm
#!pip install selenium
# para utilização do selenium é necessário baixar:
# https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
# deve ser descompactado no mesmo diretório deste notebook

In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..', 'CovidReport')))
import requests
import lxml
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urlencode
from time import sleep
import json
from tqdm import tqdm
from coletores.G1Crawler import CrawlerG1

In [3]:
def gravarJson(fileName, dados):
    with open(fileName, 'w') as outfile:
        json.dump(dados, outfile, indent=3)

def carregaDadosJson(fileName):
    with open(fileName) as json_file:
        data = json.load(json_file)
        return data

In [4]:
urlNoticia = "https://g1.globo.com/politica/blog/valdo-cruz/post/2021/03/19/nao-queremos-discurso-queremos-acao-dizem-convidados-para-reuniao-com-bolsonaro-sobre-covid.ghtml"

In [5]:
%%time 
c = CrawlerG1(urlNoticia)
noticia = c.getNoticia()
noticia["comentarios"] = c.getComentariosNoticia()

Recuperando notícia de  https://g1.globo.com/politica/blog/valdo-cruz/post/2021/03/19/nao-queremos-discurso-queremos-acao-dizem-convidados-para-reuniao-com-bolsonaro-sobre-covid.ghtml
carregando comentarios... https://g1.globo.com/politica/blog/valdo-cruz/post/2021/03/19/nao-queremos-discurso-queremos-acao-dizem-convidados-para-reuniao-com-bolsonaro-sobre-covid.ghtml
8  comentario(s) carregados
Wall time: 4.36 s


### Coleta de relação de notícias de https://g1.globo.com/sitemap/g1/sitemap.xml apenas ano 2021

In [6]:
%%time
ano = "2021"
response = requests.get("https://g1.globo.com/sitemap/g1/sitemap.xml")
xmlList = BeautifulSoup(response.content, "html.parser")
listaNoticias = []
listaBase = [i for i in xmlList.find_all("loc")]
for r in tqdm(listaBase):
    if (r.text.split("/")[5] == ano):
        response = requests.get(r.text)
        for url in BeautifulSoup(response.content, "html.parser").find_all("url"):
            loc = url.find("loc")
            regiao = ""
            localidade = ""
            if (loc): 
                loc = loc.text
                regiao = loc.split("/")[3].upper()
                localidade = loc.split("/")[4].replace("-"," ").capitalize()
            image = url.find("image:loc")
            if (image): image = image.text
            dado = {
                    "url":loc,
                    "image":image,
                    "regiao": regiao,
                    "localidade": localidade
                   }
            listaNoticias.append(dado)
print("Noticias coletadas: ", len(listaNoticias))

100%|██████████| 6941/6941 [02:16<00:00, 50.73it/s] Noticias coletadas:  85807
Wall time: 2min 19s



In [7]:
gravarJson("dados\lista-noticias-full.json", listaNoticias)

### Filtrar notícias pelas palavras chaves

In [8]:
%%time
palavrasChave = ["COVID", "PANDEMIA", "CORONAVIRUS", "VACINA"]

listaNoticiasCovid = []
contador = 0
for n in tqdm(listaNoticias):
    if any(item in palavrasChave for item in n["url"].upper().replace("/", "-").replace(".","-").split("-")):
        listaNoticiasCovid.append(n)
print("Noticias filtradas:", len(listaNoticiasCovid))    

100%|██████████| 85807/85807 [00:00<00:00, 179513.03it/s]Noticias filtradas: 21842
Wall time: 481 ms



In [9]:
gravarJson("dados\lista-noticias-covid.json", listaNoticiasCovid)

### Coletar Notícias de covid da lista

In [10]:
noticiasCovid = carregaDadosJson("dados\\noticias-covid.json")

In [11]:
auxurls = [n["url"] for n in noticiasCovid]
novas = 0
for i in listaNoticiasCovid:
    if not(i["url"] in auxurls):
        noticiasCovid.append(i)
        novas +=1
print(novas)        

22


In [12]:
%%time
c = CrawlerG1("")
c.setFeedback(False)
coletados = 0
ignorados = 0
gravar = 0
for i in tqdm(range(0,len(noticiasCovid))):
    capturar = not ("noticia" in noticiasCovid[i].keys())
    capturar = capturar or not (noticiasCovid[i]["noticia"])
    if capturar:
        try:
            c.setUrlNoticia(noticiasCovid[i]["url"])
            noticia = c.getNoticia()
            noticia["comentarios"] = c.getComentariosNoticia()
            noticiasCovid[i].update(noticia)
            coletados += 1
            gravar += 1
            if gravar >= 100:
                gravar = 0
                gravarJson("dados\\noticias-covid.json", noticiasCovid)
        except:
            ignorados += 1
            print(noticiasCovid[i]["url"])
gravarJson("dados\\noticias-covid.json", noticiasCovid)            

100%|██████████| 21843/21843 [3:14:27<00:00,  1.87it/s]
Wall time: 3h 14min 31s


In [13]:
print("coletados: ", coletados)
print("ignorados: ", ignorados)

coletados:  1761
ignorados:  0


In [14]:
coletadas = 0
for i in tqdm(range(0,len(noticiasCovid))):
    if ("noticia" in noticiasCovid[i].keys()):
        coletadas += 1
print("\n\n",coletadas, len(noticiasCovid), coletadas / len(noticiasCovid) *100)        

100%|██████████| 21843/21843 [00:00<00:00, 780363.05it/s]

 21843 21843 100.0

